# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

You should also have deployed a campaign for each of these solution versions. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover the campaigns in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the Movielens dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return a movie by just querying the ID. Movie #589 should be Terminator 2: Judgment Day.

In [5]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [7]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [8]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [9]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
Clear and Present Danger (1994)
True Lies (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Forrest Gump (1994)
Ghost (1990)
Apollo 13 (1995)
Star Trek: Generations (1994)
Cliffhanger (1993)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar movies render in a nice dataframe. Again, let's create a helper function to achieve this.

In [10]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [11]:
samples = items_df.sample(5)
samples

,title
movieId,
55207,Cashback (2004)
5304,"Rome, Open City (a.k.a. Open City) (Roma, citt..."
30803,3-Iron (Bin-jip) (2004)
143031,Jump In! (2007)
1913,Picnic at Hanging Rock (1975)


In [12]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,Cashback (2004),"Rome, Open City (a.k.a. Open City) (Roma, cittÃ aperta) (1945)",3-Iron (Bin-jip) (2004),Jump In! (2007),Picnic at Hanging Rock (1975)
0,"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)",Flatliners (1990),"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)"
1,Forrest Gump (1994),Forrest Gump (1994),"General's Daughter, The (1999)",Forrest Gump (1994),Forrest Gump (1994)
2,Pulp Fiction (1994),Pulp Fiction (1994),21 (2008),Pulp Fiction (1994),Pulp Fiction (1994)
3,"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)","Addams Family, The (1991)","Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)"
4,Braveheart (1995),Braveheart (1995),"First Wives Club, The (1996)",Braveheart (1995),Braveheart (1995)
5,"Matrix, The (1999)","Matrix, The (1999)",Bambi (1942),"Matrix, The (1999)","Matrix, The (1999)"
6,Schindler's List (1993),Schindler's List (1993),8 Mile (2002),Schindler's List (1993),Schindler's List (1993)
7,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977),"Illusionist, The (2006)",Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
8,Jurassic Park (1993),Jurassic Park (1993),Misery (1990),Jurassic Park (1993),Jurassic Park (1993)
9,Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991),101 Dalmatians (One Hundred and One Dalmatians...,Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [13]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[502, 356, 492]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [14]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [16]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,502,356,492
0,Stand by Me (1986),Anchorman: The Legend of Ron Burgundy (2004),"Cable Guy, The (1996)"
1,Robin Hood: Prince of Thieves (1991),Mean Girls (2004),Payback (1999)
2,Clerks (1994),Meet the Fockers (2004),First Strike (Police Story 4: First Strike) (G...
3,Dances with Wolves (1990),Old School (2003),Dante's Peak (1997)
4,Free Willy (1993),Napoleon Dynamite (2004),Multiplicity (1996)
5,Casper (1995),Borat: Cultural Learnings of America for Make ...,Chain Reaction (1996)
6,"City of Lost Children, The (CitÃ© des enfants ...",Dodgeball: A True Underdog Story (2004),Air Force One (1997)
7,Thelma & Louise (1991),Anger Management (2003),"Frighteners, The (1996)"
8,"Man Who Would Be King, The (1975)",Shaun of the Dead (2004),Bio-Dome (1996)
9,Free Willy 2: The Adventure Home (1995),Welcome to Mooseport (2004),Big Daddy (1999)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

### Static and Dynamic Filters

Lets interact with the static filters we created in the previous notebook, and utilize dynamic filters in realtime.

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata (that arent range based) - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.

Now lets apply item filters to see recommendations for one of these users within each decade of our static filters.


In [17]:
def get_new_recommendations_df_by_static_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [18]:
def get_new_recommendations_df_by_dynamicfilter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = genre_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given decade. Within a VOD application you could create Shelves (also known as rails or carousels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [19]:
recommendations_df_decade_shelves = pd.DataFrame()
for filter_arn in meta_filter_decade_arns:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_static_filter(recommendations_df_decade_shelves, user, filter_arn)

recommendations_df_decade_shelves

,1950s,1960s,1970s,1980s,1990s,2000s,2010s
0,Singin' in the Rain (1952),From Russia with Love (1963),Willy Wonka & the Chocolate Factory (1971),Who Framed Roger Rabbit? (1988),"Cable Guy, The (1996)",Mission: Impossible II (2000),"A-Team, The (2010)"
1,Dial M for Murder (1954),Dr. No (1962),Star Wars: Episode IV - A New Hope (1977),Dead Calm (1989),Payback (1999),"Matrix Reloaded, The (2003)",Salt (2010)
2,Rear Window (1954),Thunderball (1965),"Boys from Brazil, The (1978)",Escape from New York (1981),First Strike (Police Story 4: First Strike) (G...,Scream 3 (2000),Mission: Impossible - Ghost Protocol (2011)
3,I Confess (1953),Mary Poppins (1964),Diamonds Are Forever (1971),Indiana Jones and the Temple of Doom (1984),Dante's Peak (1997),"Matrix Revolutions, The (2003)",Sucker Punch (2011)
4,Funny Face (1957),Goldfinger (1964),Live and Let Die (1973),Star Wars: Episode VI - Return of the Jedi (1983),Multiplicity (1996),"Da Vinci Code, The (2006)","Expendables, The (2010)"
5,Harvey (1950),You Only Live Twice (1967),Deliverance (1972),Fright Night (1985),Chain Reaction (1996),"Chronicles of Narnia: The Lion, the Witch and ...",Skyfall (2012)
6,"Man Who Knew Too Much, The (1956)","Sound of Music, The (1965)","Spy Who Loved Me, The (1977)",For Your Eyes Only (1981),Air Force One (1997),Borat: Cultural Learnings of America for Make ...,Scott Pilgrim vs. the World (2010)
7,Sabrina (1954),On Her Majesty's Secret Service (1969),Moonraker (1979),Ghostbusters (a.k.a. Ghost Busters) (1984),"Frighteners, The (1996)",Pirates of the Caribbean: The Curse of the Bla...,Iron Man 2 (2010)
8,To Catch a Thief (1955),I Saw What You Did (1965),Young Frankenstein (1974),"Honey, I Shrunk the Kids (1989)",Bio-Dome (1996),Blade II (2002),Harry Potter and the Deathly Hallows: Part 1 (...
9,"Bell, Book and Candle (1958)",Topkapi (1964),"Man with the Golden Gun, The (1974)",Poltergeist (1982),Big Daddy (1999),V for Vendetta (2006),Inferno (2016)


In [20]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,0
2,Adventure|Children|Fantasy,1995,0
3,Comedy|Romance,1995,0
4,Comedy|Drama|Romance,1995,0
5,Comedy,1995,0
6,Action|Crime|Thriller,1995,0
7,Comedy|Romance,1995,0
8,Adventure|Children,1995,0
9,Action,1995,0


Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [21]:
unique_genre_field_values = items_meta_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [22]:
genres_to_filter

['Thriller',
 'IMAX',
 'Comedy',
 'Western',
 'Documentary',
 'Horror',
 'Animation',
 'Romance',
 'Children',
 'Action',
 'Musical',
 'War',
 'Drama',
 'Adventure',
 'Sci-Fi',
 'Film-Noir',
 'Fantasy',
 'Crime',
 'Mystery']

In [23]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_dynamicfilter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

,Thriller,IMAX,Comedy,Western,Documentary,Horror,Animation,Romance,Children,Action,Musical,War,Drama,Adventure,Sci-Fi,Film-Noir,Fantasy,Crime,Mystery
0,"Cable Guy, The (1996)","Matrix Reloaded, The (2003)","Cable Guy, The (1996)",Wild Wild West (1999),From the Earth to the Moon (1998),"Frighteners, The (1996)","Bug's Life, A (1998)",Tin Cup (1996),Jingle All the Way (1996),Payback (1999),"South Park: Bigger, Longer and Uncut (1999)","Peacemaker, The (1997)",Tin Cup (1996),First Strike (Police Story 4: First Strike) (G...,Lawnmower Man 2: Beyond Cyberspace (1996),Dark City (1998),"Mummy, The (1999)",Primal Fear (1996),"Thomas Crown Affair, The (1999)"
1,Payback (1999),"Matrix Revolutions, The (2003)",First Strike (Police Story 4: First Strike) (G...,Lone Star (1996),T-Rex: Back to the Cretaceous (1998),"Mummy, The (1999)","South Park: Bigger, Longer and Uncut (1999)",Runaway Bride (1999),101 Dalmatians (1996),First Strike (Police Story 4: First Strike) (G...,Hercules (1997),Three Kings (1999),8MM (1999),Chain Reaction (1996),Men in Black (a.k.a. MIB) (1997),Sin City (2005),Dragonheart (1996),Ransom (1996),Scream (1996)
2,First Strike (Police Story 4: First Strike) (G...,V for Vendetta (2006),Multiplicity (1996),Maverick (1994),"Big One, The (1997)","Relic, The (1997)",Space Jam (1996),"Nutty Professor, The (1996)",Harriet the Spy (1996),Dante's Peak (1997),Willy Wonka & the Chocolate Factory (1971),Tea with Mussolini (1999),Conspiracy Theory (1997),"Mummy, The (1999)","Nutty Professor, The (1996)",L.A. Confidential (1997),Matilda (1996),Fallen (1998),8MM (1999)
3,Dante's Peak (1997),Spider-Man 2 (2004),"Frighteners, The (1996)",Desperado (1995),Everest (1998),Scream (1996),Antz (1998),Never Been Kissed (1999),Matilda (1996),Chain Reaction (1996),Joe's Apartment (1996),"Thin Red Line, The (1998)",Primal Fear (1996),Dragonheart (1996),Star Trek: First Contact (1996),Mulholland Drive (2001),Kazaam (1996),Rush Hour (1998),Conspiracy Theory (1997)
4,Chain Reaction (1996),Night at the Museum (2006),Bio-Dome (1996),"Quick and the Dead, The (1995)",Alaska: Spirit of the Wild (1997),From Dusk Till Dawn (1996),Hercules (1997),Conspiracy Theory (1997),George of the Jungle (1997),Air Force One (1997),James and the Giant Peach (1996),Courage Under Fire (1996),"Long Kiss Goodnight, The (1996)","World Is Not Enough, The (1999)",Mars Attacks! (1996),Lost Highway (1997),"Nutty Professor, The (1996)",Striptease (1996),Primal Fear (1996)
5,Air Force One (1997),Poseidon (2006),Big Daddy (1999),Almost Heroes (1998),Forgotten Silver (1996),Stir of Echoes (1999),James and the Giant Peach (1996),Chasing Amy (1997),Kazaam (1996),"Mummy, The (1999)",Muppet Treasure Island (1996),Life Is Beautiful (La Vita Ã¨ bella) (1997),Chasing Amy (1997),101 Dalmatians (1996),"Island of Dr. Moreau, The (1996)",Suture (1993),Mystery Men (1999),"Lock, Stock & Two Smoking Barrels (1998)",Stir of Echoes (1999)
6,"Frighteners, The (1996)",Fantasia 2000 (1999),"Mummy, The (1999)",Tall Tale (1995),Hands on a Hard Body (1996),Blade (1998),Toy Story 2 (1999),Mickey Blue Eyes (1999),House Arrest (1996),Dragonheart (1996),Blues Brothers 2000 (1998),Hot Shots! Part Deux (1993),Private Parts (1997),"Ghost and the Darkness, The (1996)","Arrival, The (1996)",Devil in a Blue Dress (1995),Fallen (1998),Entrapment (1999),Mission: Impossible (1996)
7,"Mummy, The (1999)",Harry Potter and the Prisoner of Azkaban (2004),Liar Liar (1997),Purgatory (1999),Buena Vista Social Club (1999),Tales from the Crypt Presents: Bordello of Blo...,Aladdin and the King of Thieves (1996),"Wedding Singer, The (1998)",Flubber (1997),"Jackal, The (1997)",Aladdin and the King of Thieves (1996),Saving Private Ryan (1998),Eraser (1996),Anaconda (1997),Armageddon (1998),2 Days in the Valley (1996),Batman & Robin (1997),Lethal Weapon 4 (1998),"Talented Mr. Ripley, The (1999)"
8,Sleepers (1996),Beauty and the Beast (1991),Sgt. Bilko (1996),"Mariachi, El (1992)",'Hellboy': The Seeds of Creation (2004),Scream 2 (199

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [24]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:662559257807:event-tracker/3d0f73fd
c698ff69-183c-4081-badb-88ae5c06cd9d


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [25]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [26]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,492
0,"Cable Guy, The (1996)"
1,Payback (1999)
2,First Strike (Police Story 4: First Strike) (G...
3,Dante's Peak (1997)
4,Multiplicity (1996)
5,Chain Reaction (1996)
6,Air Force One (1997)
7,"Frighteners, The (1996)"
8,Bio-Dome (1996)
9,Big Daddy (1999)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [27]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [28]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for Inside (Ã l'intÃ©rieur) (2007)
sending event click for Toy, The (1982)
sending event click for Milagro Beanfield War (1988)


Now we can look at how the click events changed the recommendations.

In [29]:
user_recommendations_df

,492,Inside (Ã l'intÃ©rieur) (2007),"Toy, The (1982)",Milagro Beanfield War (1988)
0,"Cable Guy, The (1996)","Cable Guy, The (1996)","Cable Guy, The (1996)","Cable Guy, The (1996)"
1,Payback (1999),Payback (1999),Payback (1999),Ghostbusters (a.k.a. Ghost Busters) (1984)
2,First Strike (Police Story 4: First Strike) (G...,First Strike (Police Story 4: First Strike) (G...,First Strike (Police Story 4: First Strike) (G...,101 Dalmatians (1996)
3,Dante's Peak (1997),Dante's Peak (1997),Dante's Peak (1997),"Frighteners, The (1996)"
4,Multiplicity (1996),Multiplicity (1996),Multiplicity (1996),Sgt. Bilko (1996)
5,Chain Reaction (1996),Chain Reaction (1996),Chain Reaction (1996),First Strike (Police Story 4: First Strike) (G...
6,Air Force One (1997),Air Force One (1997),Air Force One (1997),Poltergeist (1982)
7,"Frighteners, The (1996)","Frighteners, The (1996)","Frighteners, The (1996)","Christmas Story, A (1983)"
8,Bio-Dome (1996),Bio-Dome (1996),Bio-Dome (1996),Bio-Dome (1996)
9,Big Daddy (1999),Big Daddy (1999),Big Daddy (1999),Chain Reaction (1996)


In the cell above, the first column after the index is the user's default recommendations from User Personalization, and each column after that has as a header of the movie that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [30]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,"Frighteners, The (1996)",Field of Dreams (1989)
1,"Nutty Professor, The (1996)",Beetlejuice (1988)
2,Jerry Maguire (1996),Dead Calm (1989)
3,Scream (1996),Who Framed Roger Rabbit? (1988)
4,Dragonheart (1996),Big (1988)
5,"Relic, The (1997)",Willy Wonka & the Chocolate Factory (1971)
6,Private Parts (1997),Scrooged (1988)
7,Bio-Dome (1996),"Craft, The (1996)"
8,Sgt. Bilko (1996),Matilda (1996)
9,Down Periscope (1996),Willow (1988)


Now lets send a watch event in for the top 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [31]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for Cable Guy, The (1996)
sending event watch for Payback (1999)
sending event watch for Multiplicity (1996)
sending event watch for Air Force One (1997)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [32]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Payback (1999),Enemy of the State (1998)
1,Dante's Peak (1997),"Jackal, The (1997)"
2,First Strike (Police Story 4: First Strike) (G...,"Thomas Crown Affair, The (1999)"
3,Air Force One (1997),Arlington Road (1999)
4,"Cable Guy, The (1996)",Conspiracy Theory (1997)
5,"Arrival, The (1996)","Glimmer Man, The (1996)"
6,Chain Reaction (1996),Breakdown (1997)
7,"Long Kiss Goodnight, The (1996)",Tomorrow Never Dies (1997)
8,"Crow: City of Angels, The (1996)","Lost World: Jurassic Park, The (1997)"
9,"Relic, The (1997)",Lawnmower Man 2: Beyond Cyberspace (1996)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [33]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [34]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,"Tale of Two Cities, A (1935)"
1,Iron Man 2 (2010)
2,Dinner for Schmucks (2010)
3,Volcano (1997)
4,Indiana Jones and the Last Crusade (1989)
5,"Closet, The (Placard, Le) (2001)"
6,Picnic (1955)
7,"Dam Busters, The (1955)"
8,"Public Enemy, The (1931)"
9,Romancing the Stone (1984)


Then make the personalized ranking API call.

In [35]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [36]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,"Tale of Two Cities, A (1935)",Volcano (1997)
1,Iron Man 2 (2010),"View to a Kill, A (1985)"
2,Dinner for Schmucks (2010),Indiana Jones and the Last Crusade (1989)
3,Volcano (1997),Lionheart (1990)
4,Indiana Jones and the Last Crusade (1989),Romancing the Stone (1984)
5,"Closet, The (Placard, Le) (2001)",Iron Will (1994)
6,Picnic (1955),He Got Game (1998)
7,"Dam Busters, The (1955)","Dam Busters, The (1955)"
8,"Public Enemy, The (1931)","Closet, The (Placard, Le) (2001)"
9,Romancing the Stone (1984),Carnival of Souls (1962)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/recommendations-batch.html). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": <S3 input path>}},
    jobOutput = 
       {"s3DataDestination": {"path": <S3 output path>}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [37]:
# We will use the same users from before
print (users)
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

[502, 356, 492]


In [38]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "502"}
{"userId": "356"}
{"userId": "492"}


Upload the file to S3 and save the path as a variable for later.

In [39]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://662559257807-us-east-1-personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [40]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://662559257807-us-east-1-personalizepocvod/


Now just make the call to kick off the batch export process.

In [41]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [42]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  06:35:37 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: ACTIVE
Import Completed on:  06:50:38 PM


In [43]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 502,User: 356,User: 492
0,Stand by Me (1986),Anchorman: The Legend of Ron Burgundy (2004),"Cable Guy, The (1996)"
1,Robin Hood: Prince of Thieves (1991),Mean Girls (2004),Payback (1999)
2,Clerks (1994),Meet the Fockers (2004),First Strike (Police Story 4: First Strike) (G...
3,Dances with Wolves (1990),Old School (2003),Dante's Peak (1997)
4,Free Willy (1993),Napoleon Dynamite (2004),Multiplicity (1996)
5,Casper (1995),Borat: Cultural Learnings of America for Make ...,Chain Reaction (1996)
6,"City of Lost Children, The (CitÃ© des enfants ...",Dodgeball: A True Underdog Story (2004),Air Force One (1997)
7,Thelma & Louise (1991),Anger Management (2003),"Frighteners, The (1996)"
8,"Man Who Would Be King, The (1975)",Shaun of the Dead (2004),Bio-Dome (1996)
9,Free Willy 2: The Adventure Home (1995),Welcome to Mooseport (2004),Big Daddy (1999)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, make a pull request and provide any additional helpful components that may be missing from this collection.

You can choose to head to `04_Operations_Layer.ipynb` to go deeper into ML Ops and what a production solution can look like with an automation pipeline.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `05_Clean_Up.ipynb`.

In [44]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)
